Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Lars Janssen"

---

For those not familiar with Python, a quick overview is given [here](https://github.com/palcu/python-for-competitive-programming/blob/master/python-for-competitive-programming.ipynb).

# Week 12 - Dijkstra

## Implementation
Dijkstra's algorithm is a way to find the shortest paths from a 'source' to all other vertices (or a specific target vertex). The algorithm works for _directed_ and _undirected_ graphs, where the edges have a specific weight. Let's consider the following graph:

![Image of Yaktocat](https://cdncontribute.geeksforgeeks.org/wp-content/uploads/dijikstra.png)

The code below gives an implementation of Dijkstra, using a priorty queue to find the currenty closest vertex ([this pseudocode](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Using_a_priority_queue)). Here
we used an adjacency list representation of this graph: we store an edge as a pair $(v, w)$ where $v$ is the connecting vertex, and $w$ is the weight.

In [2]:
from heapq import heappush, heappop

# Adjacency list of the graph above, an edge is described by the 
# pair (v, w) -- the other vertex, weight of the edge.
adj = [[(1, 4), (7, 8)], 
       [(0, 4), (7, 11), (2, 8)], 
       [(1, 8), (3, 7), (5, 4), (8, 2)], 
       [(2, 7), (4, 9), (5, 14)], 
       [(3, 9), (5, 10)], 
       [(2, 4), (3, 14), (4, 10), (6, 2)], 
       [(5, 2), (8, 6), (7, 1)], 
       [(0, 8), (1, 11), (6, 1), (8, 7)], 
       [(2, 2), (6, 6), (7, 7)]]
n = len(adj)

# Initialize the distance to every vertex to `infinite`.
dist = [10**10 for _ in range(n)]

# Recall that a priority queue is just a list in Python.
Q = []

# Initialize the PQ with source.
source = 0
# We are going to put pairs (distance, node) on the priority queue:
# this ensures that the node with lowest distance is popped first.
heappush(Q, (0, source))
dist[source] = 0 

while len(Q):
    dist_u, u = heappop(Q) # Retrieve node with tot dist d.
    
    if dist[u] < dist_u: # We've already handled this node with lower distance.
        continue
    
    for v, w in adj[u]:
        # Relaxing the distance:
        if dist[v] > dist[u] + w:
            dist[v] = dist[u] + w
            heappush(Q, (dist[v], v))

for u in range(n):
    print('Dist from {} to {} is {}'.format(source, u, dist[u]))

Dist from 0 to 0 is 0
Dist from 0 to 1 is 4
Dist from 0 to 2 is 12
Dist from 0 to 3 is 19
Dist from 0 to 4 is 21
Dist from 0 to 5 is 11
Dist from 0 to 6 is 9
Dist from 0 to 7 is 8
Dist from 0 to 8 is 14


## Excercise 1: Single-Source Shortest Path
The above code does not calculate the shortest _path_, only its length. Below, write a function which takes a graph and two nodes, and outputs the shortest path between those nodes as a list.

In [50]:
from heapq import heappush, heappop

def shortest_path(adj, source, target):
    """
    Compute the shortest path from node `source` to node `target` 
    in weighted graph `adj`.
    """
    n = len(adj)
    dist = [10**10 for _ in range(n)]
    previous = [None for _ in range(n)]
    Q = []
    
    heappush(Q, (0, source))
    dist[source] = 0 

    while len(Q):
        dist_u, u = heappop(Q)
        if(u == target):
            break

        if dist[u] < dist_u:
            continue

        for v, w in adj[u]:
            if dist[v] > dist[u] + w:
                dist[v] = dist[u] + w
                previous[v] = u
                heappush(Q, (dist[v], v))
    
    S = []
    u = target
    if(previous[u] != None or u == source):
        while u != None:
            S.append(u)
            u = previous[u]
    S.reverse()
    return S

In [51]:
adj = [[(1, 4), (7, 8)], 
       [(0, 4), (7, 11), (2, 8)], 
       [(1, 8), (3, 7), (5, 4), (8, 2)], 
       [(2, 7), (4, 9), (5, 14)], 
       [(3, 9), (5, 10)], 
       [(2, 4), (3, 14), (4, 10), (6, 2)], 
       [(5, 2), (8, 6), (7, 1)], 
       [(0, 8), (1, 11), (6, 1), (8, 7)], 
       [(2, 2), (6, 6), (7, 7)]]

assert shortest_path(adj, 0, 8) == [0,1,2,8]
assert shortest_path(adj, 0, 4) == [0, 7, 6, 5, 4]
assert shortest_path(adj, 1, 6) == [1,7,6]
assert shortest_path(adj, 8, 4) == [8,2,5,4]

## Exercise 2: SSSP in a DAG

Dijkstra's algorithm works pretty fast, but in special cases we can be even faster. If the graph is a DAG, we can actually find the shortest paths from a given node in $\mathcal O(E)$ time. Write the following function which computes distances in a DAG in $\mathcal O(E)$ time. For simplicity, you may assume that the graph is already given in toposort order. That is, if there is an edge $v \to w$, then you may assume that $v < w$.

In [58]:
def DAG_distance(adj, source, target):
    """
    Return the length of the shortest path from source to target in adj,
    or return None if no such path exists. You may assume that adj is a
    DAG given in toposort order.
    """
    n = len(adj)

    # Initialize the distance to every vertex to `infinite`.
    dist = [10**10 for _ in range(n)]

    # Recall that a priority queue is just a list in Python.
    Q = []

    # Initialize the PQ with source.
    # We are going to put pairs (distance, node) on the priority queue:
    # this ensures that the node with lowest distance is popped first.
    heappush(Q, (0, source))
    dist[source] = 0 

    while len(Q):
        dist_u, u = heappop(Q) # Retrieve node with tot dist d.
        if(u == target):
            break
        for v, w in adj[u]:
            # Relaxing the distance:
            if dist[v] > dist[u] + w:
                dist[v] = dist[u] + w
                heappush(Q, (dist[v], v))
    if(dist[target] != 10**10):
        return dist[target]
    return None

In [59]:
adj = [[(2, 5), (3, 6), (4, 12), (5, 15)], [(3, 10), (6, 11)],
      [(4, 3), (5, 2)], [(5, 2)], [], [(6, 1)], []]

assert DAG_distance(adj, 0, 5) == 7
assert DAG_distance(adj, 0, 1) == None
assert DAG_distance(adj, 1, 6) == 11
assert DAG_distance(adj, 0, 4) == 8
assert DAG_distance(adj, 6, 6) == 0
assert DAG_distance(adj, 2, 5) == 2
assert DAG_distance(adj, 5, 2) == None

## Exercise 3: Dijkstra with a twist

In programming contests, a Dijkstra exercise will often have some kind of twist, to keep it interesting. Let's look at one example: [Get Shorty](https://open.kattis.com/problems/getshorty) on Kattis. Here we get a graph, and we need to go from one node to another node, but the edges don't have ordinary distances. Instead, each edge has a _shrink ray_: if we traverse an edge, we are shrunk by some factor $0 < f \leq 1$. We want to traverse the graph, and come out as tall as possible.

You have to think about how this relates to the Dijkstra algorithm. There are at least two approaches:

1. Think about how the Dijkstra algorithm works. Normally, you want to minimize something (the sum of the weights of the edges on your path). Now you want to maximize the product of those weights. What do you have to change? How do you ensure that your priority queue pops the right node to relax?
2. Can you somehow transform the problem so that you can solve it with an ordinary Dijkstra algorithm, and still produce the correct output?

In [94]:
def get_shorty(adj, source, target):
    """
    Given a weighted graph adj, whose weights are all between 0 and 1,
    find the path from source to target for which the product of the
    weights is as big as possible, and return the product of the weights
    on that path.
    
    Note that for the Kattis problem, the source is always 0, and the
    target is always n-1, and that you have to return the correct result 
    rounded to exactly four decimal places -- you don't need to do that
    in this function.
    """
    n = len(adj)
    Q = []
    dist = [0 for u in range(n)]
    dist[source] = 1
    heappush(Q, (-1,source))
    while len(Q):
        dist_u, u = heappop(Q)
        dist_u = -dist_u
        if(u == target):
            break
        if(dist[u] > dist_u):
            continue
        for (v,w) in adj[u]:
            if dist[v] < dist[u] * w:
                dist[v] = dist[u] * w
                heappush(Q, (-dist[v], v))
    return dist[target]

In [95]:
import math

# The example from Kattis.
adj = [[(1, 0.9), (2, 0.8)], [(2, 0.9)], []]
assert math.isclose(get_shorty(adj, 0, 2), 0.81)

adj = [[(1, 1/4), (7, 1/8)], 
       [(0, 1/4), (7, 1/11), (2, 1/8)], 
       [(1, 1/8), (3, 1/7), (5, 1/4), (8, 1/2)], 
       [(2, 1/7), (4, 1/9), (5, 1/14)], 
       [(3, 1/9), (5, 1/10)], 
       [(2, 1/4), (3, 1/14), (4, 1/10), (6, 1/2)], 
       [(5, 1/2), (8, 1/6), (7, 1/1)], 
       [(0, 1/8), (1, 1/11), (6, 1/1), (8, 1/7)], 
       [(2, 1/2), (6, 1/6), (7, 1/7)]]
assert math.isclose(get_shorty(adj, 2, 5), 0.25)
assert math.isclose(get_shorty(adj, 0, 8), 0.020833333333)
assert math.isclose(get_shorty(adj,0, 4), 0.00625)
assert math.isclose(get_shorty(adj,0, 6), 0.125)
assert math.isclose(get_shorty(adj,1, 4), 0.00454545454545)